# Demographic Data Cleanup and Export

see https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-detail.html#par_textimage_1383669527

see input_data/county_demographics directory for .pdf which contains column definitions for csv

In [1]:
import pandas as pd
import numpy as np
import sqlalchemy as sq
import datetime as dt
import warnings
import os

import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as py
from plotly.subplots import make_subplots

warnings.filterwarnings(action="once")
np.set_printoptions(suppress=True)
pd.set_option("display.float_format", lambda x: "%.5f" % x)
pd.options.display.max_rows = 999

In [2]:
date = dt.datetime.today().strftime("%m_%d_%Y")
print(date)

05_03_2020


In [3]:
df = pd.read_csv('input_data/county_demographics/cc-est2018-alldata.csv', encoding = "ISO-8859-1")
print(df.shape)

(656678, 80)


In [4]:
df.head(2)

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
0,50,1,1,Alabama,Autauga County,1,0,54571,26569,28002,...,607,538,57,48,26,32,9,11,19,10
1,50,1,1,Alabama,Autauga County,1,1,3579,1866,1713,...,77,56,9,5,4,1,0,0,2,1


In [5]:
df.columns

Index(['SUMLEV', 'STATE', 'COUNTY', 'STNAME', 'CTYNAME', 'YEAR', 'AGEGRP',
       'TOT_POP', 'TOT_MALE', 'TOT_FEMALE', 'WA_MALE', 'WA_FEMALE', 'BA_MALE',
       'BA_FEMALE', 'IA_MALE', 'IA_FEMALE', 'AA_MALE', 'AA_FEMALE', 'NA_MALE',
       'NA_FEMALE', 'TOM_MALE', 'TOM_FEMALE', 'WAC_MALE', 'WAC_FEMALE',
       'BAC_MALE', 'BAC_FEMALE', 'IAC_MALE', 'IAC_FEMALE', 'AAC_MALE',
       'AAC_FEMALE', 'NAC_MALE', 'NAC_FEMALE', 'NH_MALE', 'NH_FEMALE',
       'NHWA_MALE', 'NHWA_FEMALE', 'NHBA_MALE', 'NHBA_FEMALE', 'NHIA_MALE',
       'NHIA_FEMALE', 'NHAA_MALE', 'NHAA_FEMALE', 'NHNA_MALE', 'NHNA_FEMALE',
       'NHTOM_MALE', 'NHTOM_FEMALE', 'NHWAC_MALE', 'NHWAC_FEMALE',
       'NHBAC_MALE', 'NHBAC_FEMALE', 'NHIAC_MALE', 'NHIAC_FEMALE',
       'NHAAC_MALE', 'NHAAC_FEMALE', 'NHNAC_MALE', 'NHNAC_FEMALE', 'H_MALE',
       'H_FEMALE', 'HWA_MALE', 'HWA_FEMALE', 'HBA_MALE', 'HBA_FEMALE',
       'HIA_MALE', 'HIA_FEMALE', 'HAA_MALE', 'HAA_FEMALE', 'HNA_MALE',
       'HNA_FEMALE', 'HTOM_MALE', 'HTOM_FEMALE

In [6]:
want_columns = ['SUMLEV', 'STATE', 'COUNTY', 'STNAME', 'CTYNAME', 'YEAR', 'AGEGRP',
       'TOT_POP', 'TOT_MALE', 'TOT_FEMALE', 'WAC_MALE', 'WAC_FEMALE',
                'BAC_MALE', 'BAC_FEMALE', 'AAC_MALE', 'AAC_FEMALE',
               'H_MALE', 'H_FEMALE']

In [7]:
df2 = df[want_columns].reset_index(drop=True)
print(df2.shape)

(656678, 18)


In [8]:
df2['WHITE_POP_PERC'] = (df2['WAC_MALE'] + df2['WAC_FEMALE']) / df2['TOT_POP']
df2['BLACK_POP_PERC'] = (df2['BAC_MALE'] + df2['BAC_FEMALE']) / df2['TOT_POP']
df2['ASIAN_POP_PERC'] = (df2['AAC_MALE'] + df2['AAC_FEMALE']) / df2['TOT_POP']
df2['HISP_POP_PERC'] = (df2['H_MALE'] + df2['H_FEMALE']) / df2['TOT_POP']
df2['MALE_PERC'] = df2['TOT_MALE'] / df2['TOT_POP']
df2['FEMALE_PERC'] = df2['TOT_FEMALE'] / df2['TOT_POP']

In [9]:
df3 = df2[['STATE', 'COUNTY', 'STNAME', 'CTYNAME', 'YEAR', 'AGEGRP',
       'TOT_POP', 'MALE_PERC', 'FEMALE_PERC','WHITE_POP_PERC', 'BLACK_POP_PERC',
          'ASIAN_POP_PERC', 'HISP_POP_PERC']].reset_index(drop=True)

In [10]:
df3['STATE_COUNTY'] = df3['STATE'].astype('str') + '_' + df3['COUNTY'].astype('str')

In [11]:
df4 = df3[df3.YEAR == 11].reset_index(drop=True)
print(df4.shape)

(59698, 14)


In [12]:
df4.STATE_COUNTY.nunique()

3142

In [13]:
df4.head(2)

,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,MALE_PERC,FEMALE_PERC,WHITE_POP_PERC,BLACK_POP_PERC,ASIAN_POP_PERC,HISP_POP_PERC,STATE_COUNTY
0,1,1,Alabama,Autauga County,11,0,55601,0.48551,0.51449,0.78380,0.20617,0.01730,0.02966,1_1
1,1,1,Alabama,Autauga County,11,1,3364,0.51367,0.48633,0.76546,0.25149,0.02556,0.04875,1_1


In [14]:
df5 = df4[df4.AGEGRP == 0].reset_index(drop=True)
df5.shape

(3142, 14)

## Calculate Age Bins

In [16]:
age_df = df4[df4.AGEGRP != 0].reset_index(drop=True)
age_df = age_df[['STATE_COUNTY', 'AGEGRP', 'TOT_POP']]

In [17]:
age_df2 = age_df.pivot_table(
    index="STATE_COUNTY", columns=["AGEGRP"], values=["TOT_POP"]
)

In [18]:
age_df2 = age_df2.reset_index().rename_axis([None, None], axis=1)

In [19]:
age_df2.columns = [
    "STATE_COUNTY", "A1", "A2", "A3", "A4", "A5", "A6", "A7", "A8", "A9", "A10", "A11",
    "A12", "A13", "A14", "A15", "A16", "A17", "A18"
]

In [20]:
age_df2.head(2)

,STATE_COUNTY,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,A18
0,10_1,11170,11264,11516,12105,12492,12835,11499,10898,9841,10345,11305,11933,10864,9479,8345,5877,3631,3151
1,10_3,32142,33133,33905,37412,35542,42067,39244,36710,32417,35910,38448,39526,35851,28539,22280,15297,9978,10934


## Merge Back in Calc Age Bin into Final DF

In [21]:
df6 = pd.merge(df5, age_df2, on=['STATE_COUNTY'])
print(df6.shape)

(3142, 32)


In [22]:
df6.head(2)

,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,MALE_PERC,FEMALE_PERC,WHITE_POP_PERC,...,A9,A10,A11,A12,A13,A14,A15,A16,A17,A18
0,1,1,Alabama,Autauga County,11,0,55601,0.48551,0.51449,0.78380,...,3616,3966,3811,3927,3237,2589,2311,1739,1143,871
1,1,3,Alabama,Baldwin County,11,0,218022,0.48462,0.51538,0.88960,...,12877,14461,14242,15744,15665,14267,12471,8465,5064,4304


In [23]:
df6['AGE_OTO4'] = df6['A1']
df6['AGE_5TO14'] = df6['A2'] + df6['A3']
df6['AGE_15TO24'] = df6['A4'] + df6['A5']
df6['AGE_25TO34'] = df6['A6'] + df6['A7']
df6['AGE_35TO44'] = df6['A8'] + df6['A9']
df6['AGE_45TO54'] = df6['A10'] + df6['A11']
df6['AGE_55TO64'] = df6['A12'] + df6['A13']
df6['AGE_65TO74'] = df6['A14'] + df6['A15']
df6['AGE_75TO84'] = df6['A16'] + df6['A17']
df6['AGE_84PLUS'] = df6['A18']

In [24]:
for col in ["A1", "A2", "A3", "A4", "A5", "A6", "A7", "A8", "A9", "A10", "A11",
    "A12", "A13", "A14", "A15", "A16", "A17", "A18"]:
    del df6[col]

In [25]:
del df6['AGEGRP']
del df6['YEAR']
del df6['STATE_COUNTY']

In [26]:
df6.shape

(3142, 21)

In [27]:
df6.columns

Index(['STATE', 'COUNTY', 'STNAME', 'CTYNAME', 'TOT_POP', 'MALE_PERC',
       'FEMALE_PERC', 'WHITE_POP_PERC', 'BLACK_POP_PERC', 'ASIAN_POP_PERC',
       'HISP_POP_PERC', 'AGE_OTO4', 'AGE_5TO14', 'AGE_15TO24', 'AGE_25TO34',
       'AGE_35TO44', 'AGE_45TO54', 'AGE_55TO64', 'AGE_65TO74', 'AGE_75TO84',
       'AGE_84PLUS'],
      dtype='object')

In [28]:
df6.columns = ['state_fips', 'county_fips', 'state', 'county', 'TOT_POP',
       'MALE_PERC', 'FEMALE_PERC', 'WHITE_POP_PERC', 'BLACK_POP_PERC',
       'ASIAN_POP_PERC', 'HISP_POP_PERC', 'AGE_OTO4',
       'AGE_5TO14', 'AGE_15TO24', 'AGE_25TO34', 'AGE_35TO44', 'AGE_45TO54',
       'AGE_55TO64', 'AGE_65TO74', 'AGE_75TO84', 'AGE_84PLUS']

In [29]:
df6.head()

,state_fips,county_fips,state,county,TOT_POP,MALE_PERC,FEMALE_PERC,WHITE_POP_PERC,BLACK_POP_PERC,ASIAN_POP_PERC,...,AGE_OTO4,AGE_5TO14,AGE_15TO24,AGE_25TO34,AGE_35TO44,AGE_45TO54,AGE_55TO64,AGE_65TO74,AGE_75TO84,AGE_84PLUS
0,1,1,Alabama,Autauga County,55601,0.48551,0.51449,0.78380,0.20617,0.01730,...,3364,7305,6928,7166,7244,7777,7164,4900,2882,871
1,1,3,Alabama,Baldwin County,218022,0.48462,0.51538,0.88960,0.09688,0.01561,...,11955,26783,24040,24528,26033,28703,31409,26738,13529,4304
2,1,5,Alabama,Barbour County,24881,0.52783,0.47217,0.50034,0.49291,0.00607,...,1284,2957,2923,3466,2981,3227,3211,2877,1483,472
3,1,7,Alabama,Bibb County,22400,0.53219,0.46781,0.77812,0.21902,0.00379,...,1285,2497,2573,3327,2952,3179,2897,2174,1153,363
4,1,9,Alabama,Blount County,57840,0.49274,0.50726,0.97184,0.02163,0.00541,...,3460,7537,6846,6934,7067,7818,7630,6243,3273,1032


In [30]:
df6.to_csv('us_county_demographics.csv', index=False)